### Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd

import plotly.graph_objects as go
import plotly.io as pio


from log_parser import *
from odrive_utils import *
from figures import *

### Get Logs

In [ ]:
# config
log_dir = "logs"
generate_html = True
offline = False
show_figures = False
min_log_size_kb = 1
csv_logs = False
dark_theme = True

In [ ]:
# get paths
if csv_logs:
    log_ext = "txt"
    logParserFunc = parseCSVFile
else:
    log_ext = "bin"
    logParserFunc = parseBinaryFile
if dark_theme:
    pio.templates.default = "plotly_dark"
else:
    pio.templates.default = "plotly_white"


paths = getLogsByExtension(log_dir, log_ext)
paths = filterFilesBySize(paths, min_log_size_kb)

In [ ]:
# create dataframes
dfs = []

for path in paths:
    _, df = logParserFunc(path)
    if df is None:
        continue
    postProcessDataframe(df)
    df.filename = os.path.basename(path)
    addNormalizedColumns(df)
    dfs.append(df)

### Create per Log Graphs

In [ ]:
# create graphs
figure_funcs = [
    getRPMFigure,
    # getRPMAndActuatorFigure,
    getVehicleSpeedFigure,
    # getShiftRatioAndAcuatorFigure,
    # getShiftRatioFigure,
    # getVelocityCommandFigure,
    # getShadowCountFigure,
    # getEngineVsWheelFigure,
    # getEverythingFigure,
    # getVoltageAndCurrentFigure,
    getCarPositionFigure,
]

all_figs_by_log = []

for df in dfs:
    figs = []
    for figure_func in figure_funcs:
        figs.append(figure_func(df))
    all_figs_by_log.append(figs)

if generate_html:
    for path, figs_by_log in zip(paths, all_figs_by_log):
        filename_without_ext = os.path.splitext(os.path.basename(path))[0]
        html_path = f"graphs/{filename_without_ext}.html"
        figuresToHTML(figs_by_log, html_path, offline=offline, dark_theme=dark_theme)

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output

show_figures = True
if show_figures:
    dropdown = widgets.Dropdown(options=paths, value=paths[-1])

    def onDropdownChange(change):
        if change["type"] == "change" and change["name"] == "value":
            clear_output()
            idx = paths.index(change["new"])
            printErrorTimes(dfs[idx], ErrorType.MOTOR)
            display(dropdown)
            for fig in all_figs_by_log[idx]:
                fig.show()

    idx = -1
    dropdown.observe(onDropdownChange)
    printErrorTimes(dfs[idx], ErrorType.MOTOR)
    display(dropdown)
    for fig in all_figs_by_log[idx]:
        fig.show()
print(np.max(df["control_cycle_start_s"]))

### Create Group Graphs (By Graph Type)

In [ ]:
"""
figure_names_and_funcs = [
    ("rpm", getRPMFigure),
    ("rpm-and-actuator", getRPMAndActuatorFigure),
    ("vehicle-speed", getVehicleSpeedFigure),
    ("shift-ratio-and-acuator", getShiftRatioAndAcuatorFigure),
    ("velocity-command", getVelocityCommandFigure),
    ("shadow-count", getShadowCountFigure),
    ("shift-ratio", getShiftRatioFigure),
    ("engine-vs-wheel", getEngineVsWheelFigure),
]
all_figs_by_type = []
for name, func in figure_names_and_funcs:
    figs_by_type = []
    for df in dfs:
        figs_by_type.append(func(df))
    all_figs_by_type.append((name, figs_by_type))

if generate_html:
    for figs_by_type in all_figs_by_type:
        filename_without_ext = os.path.splitext(os.path.basename(path))[0]
        html_path = f"graphs/all-{figs_by_type[0]}.html"
        figuresToHTML(figs_by_type[1], html_path, offline=offline)
"""

# Scratch Work

In [ ]:
header, df = parseBinaryFile("logs/log_2023-04-23_18-30-50.bin")
max_time_idx = df["control_cycle_start_us"].argmax()
if max_time_idx != len(df) - 1:
    dts = np.cumsum(df["control_cycle_dt_us"].iloc[max_time_idx + 1 :])

    overflow_start_time_us = df["control_cycle_start_us"].iloc[max_time_idx]
    overflow_stop_time_us = df["control_cycle_stop_us"].iloc[max_time_idx]
    df["control_cycle_start_us"].iloc[max_time_idx + 1 :] = df[
        "control_cycle_start_us"
    ].iloc[max_time_idx + 1 :] + (overflow_start_time_us + dts)
    df["control_cycle_stop_us"].iloc[max_time_idx + 1 :] = df[
        "control_cycle_stop_us"
    ].iloc[max_time_idx + 1 :] + (overflow_stop_time_us + dts)
dumpToBinary("out.bin", header, df)

In [ ]:
def winter_low(cutoff_freq, sample_time, x0, x1, x2, y1, y2, print_coeff=False):
    """Filters a data sample based on two past unfiltered and filtered data samples.

    2nd order low pass, single pass butterworth filter presented in Winter2009.

    Parameters
    ==========
    cuttoff_freq: float
        The desired lowpass cutoff frequency in Hertz.
    sample_time: floaat
        The difference in time between the current time and the previous time.
    x0 : float
        The current unfiltered signal, x_i
    x1 : float
        The unfiltered signal at the previous sampling time, x_i-1.
    x2 : float
        The unfiltered signal at the second previous sampling time, x_i-2.
    y1 : float
        The filtered signal at the previous sampling time, y_i-1.
    y2 : float
        The filtered signal at the second previous sampling time, y_i-2.

    """
    sampling_rate = 1 / sample_time  # Hertz

    correction_factor = 1.0  # 1.0 for a single pass filter

    corrected_cutoff_freq = (
        np.tan(np.pi * cutoff_freq / sampling_rate) / correction_factor
    )  # radians

    K1 = np.sqrt(2) * corrected_cutoff_freq
    K2 = corrected_cutoff_freq**2

    a0 = K2 / (1 + K1 + K2)
    a1 = 2 * a0
    a2 = a0

    K3 = a1 / K2

    b1 = -a1 + K3
    b2 = 1 - a1 - K3

    if print_coeff:
        print("num:", a0, a1, a2)
        print("dem:", 1.0, -b1, -b2)

    return a0 * x0 + a1 * x1 + a2 * x2 + b1 * y1 + b2 * y2

In [ ]:
import plotly.express as px

In [ ]:
df = dfs[0]
df = trimDataFrame(df, start_s=37, end_s=160)
dt_s = np.array(df["control_cycle_dt_us"]) / 1e6
t_s = np.array(df["control_cycle_start_us"]) / 1e6
sd_rpm = np.array(df["secondary_rpm"])
filt_sd_rpm = np.zeros_like(sd_rpm)

for i in range(2, len(t_s)):
    filt_sd_rpm[i] = winter_low(
        0.8,
        dt_s[i],
        sd_rpm[i],
        sd_rpm[i - 1],
        sd_rpm[i - 2],
        filt_sd_rpm[i - 1],
        filt_sd_rpm[i - 2],
    )

px.line(x=t_s, y=[sd_rpm, filt_sd_rpm])

In [ ]:
df = dfs[0]
df = trimDataFrame(df, start_s=37, end_s=160)
df["secondary_freq_mag"] = np.abs(np.fft.fft(df["secondary_rpm"]))
df["engine_freq_mag"] = np.abs(np.fft.fft(df["engine_rpm"]))
df["freqs"] = np.fft.fftfreq(len(df), np.mean(df["control_cycle_dt_us"] / 1e6))

# px.line(df, x="control_cycle_start_s", y="secondary_rpm")
px.line(df, x="freqs", y="engine_freq_mag")

In [ ]:
df = dfs[0]
dt_s = np.array(df["control_cycle_dt_us"]) / 1e6
t_s = np.array(df["control_cycle_start_us"]) / 1e6
eg_rpm = np.array(df["engine_rpm"])
filt_eg_rpm = np.zeros_like(eg_rpm)
exp_filt_eg_rpm = np.zeros_like(eg_rpm)
vel_cmd = np.array(df["velocity_command"])
deriv_vel_cmd = np.array(df["velocity_command"])

alpha = 0.7
for i in range(2, len(t_s)):
    filt_eg_rpm[i] = winter_low(
        1.2,
        dt_s[i],
        eg_rpm[i],
        eg_rpm[i - 1],
        eg_rpm[i - 2],
        filt_eg_rpm[i - 1],
        filt_eg_rpm[i - 2],
    )
    exp_filt_eg_rpm[i] = alpha * eg_rpm[i] + (1 - alpha) * exp_filt_eg_rpm[i - 1]

d_eg_rpm = np.diff(eg_rpm) / dt_s[1:]
d_filt_eg_rpm = np.diff(filt_eg_rpm) / dt_s[1:]
KD = 0.01
for i in range(len(t_s) - 1):
    deriv_vel_cmd[i] = vel_cmd[i] + d_filt_eg_rpm[i] * KD

d_exp_filt_eg_rpm = np.diff(exp_filt_eg_rpm) / dt_s[1:]
px.line(x=t_s, y=[eg_rpm, filt_eg_rpm])
# px.line(x=t_s[1:], y=[d_eg_rpm, d_filt_eg_rpm,d_exp_filt_eg_rpm])

In [ ]:
px.line(df, x="control_cycle_start_s", y="engine_count")